# Explore here

It's recommended to use this notebook for exploration purposes.

For example: 

1. You could import the CSV generated by python into your notebook and explore it.
2. You could connect to your database using `pandas.read_sql` from this notebook and explore it.

In [38]:
# import packages

import pandas as pd
import requests
from bs4 import BeautifulSoup
import sqlite3
import time

In [39]:
url = " https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"
html_data = requests.get(url, time.sleep(10)).text

In [40]:
soup = BeautifulSoup(html_data,"html.parser")

In [42]:
#find all tables
tables = soup.find_all('table')

for index,table in enumerate(tables):
    if ("Tesla Quarterly Revenue" in str(table)):
        table_index = index

#create a dataframe        
tesla_revenue = pd.DataFrame(columns=["Date", "Revenue"])

for row in tables[table_index].tbody.find_all("tr"):
    col = row.find_all("td")
    if (col != []):
        Date = col[0].text
        Revenue = col[1].text.replace("$", "").replace(",", "")
        tesla_revenue = tesla_revenue.append({"Date":Date, "Revenue":Revenue}, ignore_index=True)


#using read_html

#tesla_revenue=pd.read_html(url, match="Tesla Quarterly Revenue", flavor='bs4')[0]
#tesla_revenue = tesla_revenue.rename(columns={"Tesla Quarterly Revenue(Millions of US $)":"Date","Tesla Quarterly Revenue(Millions of US $).1":"Revenue"}) #Rename df columns to 'Date' and 'Revenue'
#tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(',|\$',"") # remove the comma and dollar sign from the 'Revenue' column
#tesla_revenue.head()

TypeError: list indices must be integers or slices, not NoneType

In [29]:
tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]
tesla_revenue

,Date,Revenue


In [30]:
type(tesla_revenue)

pandas.core.frame.DataFrame

In [32]:
pd.core.frame.DataFrame

pandas.core.frame.DataFrame

In [33]:
records = tesla_revenue.to_records(index=False)
list_of_tuples = list(records)
list_of_tuples

[]

In [34]:
# Use the connect() function of sqlite3 to create a database. It will create a connection object.

connection = sqlite3.connect('Tesla.db')

In [35]:
c = connection.cursor()

# Create table
c.execute('''CREATE TABLE revenue
             (Date, Revenue)''')

In [36]:
# Insert the values
c.executemany('INSERT INTO revenue VALUES (?,?)', list_of_tuples)
# Save (commit) the changes
connection.commit()

In [37]:
for row in c.execute('SELECT * FROM revenue'):
    print(row)